# Carga del dataset *Climate Model Simulation Crashes*

Para comenzar con esta actividad, realizamos la carga del *Climate Model Simulation Crashes Data Set* disponible en la plataforma *UCI Machine Learning Repository* (https://archive.ics.uci.edu/ml/datasets/climate+model+simulation+crashes)

Comenzamos realizando la carga mediante ``pandas``. En este caso, el archivo proporcionado contiene una cabecera con los identificadores de cada columna. Sin embargo, el fichero presentaba una dificultad adicional y es que cada columna estaba separada por un número aparentemente arbitrario de espacios, lo que complicaba su lectura. Para solventar este error, se abrió dicho fichero desde *VS Code* y, mediante el uso de la expresión regular " +", se reemplazaron todos los espacios por una coma.

In [1]:
import pandas as pd

filename = 'pop_failures.dat'
df = pd.read_csv(filename)
df.head()

,Study,Run,vconst_corr,vconst_2,vconst_3,vconst_4,vconst_5,vconst_7,ah_corr,ah_bolus,...,efficiency_factor,tidal_mix_max,vertical_decay_scale,convect_corr,bckgrnd_vdc1,bckgrnd_vdc_ban,bckgrnd_vdc_eq,bckgrnd_vdc_psim,Prandtl,outcome
0,1,1,0.859036,0.927825,0.252866,0.298838,0.170521,0.735936,0.428325,0.567947,...,0.245675,0.104226,0.869091,0.997518,0.448620,0.307522,0.858310,0.796997,0.869893,0
1,1,2,0.606041,0.457728,0.359448,0.306957,0.843331,0.934851,0.444572,0.828015,...,0.616870,0.975786,0.914344,0.845247,0.864152,0.346713,0.356573,0.438447,0.512256,1
2,1,3,0.997600,0.373238,0.517399,0.504993,0.618903,0.605571,0.746225,0.195928,...,0.679355,0.803413,0.643995,0.718441,0.924775,0.315371,0.250642,0.285636,0.365858,1
3,1,4,0.783408,0.104055,0.197533,0.421837,0.742056,0.490828,0.005525,0.392123,...,0.471463,0.597879,0.761659,0.362751,0.912819,0.977971,0.845921,0.699431,0.475987,1
4,1,5,0.406250,0.513199,0.061812,0.635837,0.844798,0.441502,0.191926,0.487546,...,0.551543,0.743877,0.312349,0.650223,0.522261,0.043545,0.376660,0.280098,0.132283,1


# Conteo de clases y desequilibrado

Procedemos a hacer un conteo de las diferentes clases en el dataset haciendo uso del método ``.value_counts()``.

In [2]:
df['outcome'].value_counts()

1    494
0     46
Name: outcome, dtype: int64

Como podemos observar, el dataset cuenta con un desequilibrado notable de los datos. En concreto, contamos con 540 datos, el ~91.48% etiquetados como "1" y el resto como "0".

# Técnicas de validación cruzada con KNeighbors

Vamos a proceder a analizar el rendimiento del modelo *k-Nearest Neighbors* utilizando los siguientes métodos de validación cruzada:

1.- Validación con *K-folds*

2.- Validación con *Leave-One-Out*

3.- Validación con *Monte Carlo*

Al tratarse de un problema de clasificación binaria, utilizaremos métricas de clasificación para determinar el rendimiento obtenido. En concreto, al tratarse de un problema desequilibrado descartamos el uso de la *accuracy* como métrica del rendimento. Como vimos en sesiones de teoría, el *accuracy* puede llevar a confusión cuando nuestro dataset tiene un número no equitativo de muestras en cada clase. Por ejemplo, supongamos un dataset con 1000 instancias, 990 de las cuales forman parte de la clase "1" y 10 de la clase "0" y un modelo que, ante cualquier entrada, devuelve "1" como la clase predicha. Este modelo arrojaría un valor del 99% de *accuracy* a pesar de que clasifica de forma incorrecta todas las instancias de clase "0". En general, la tendencia de modelos entrenados con datasets desequilibrados es que se favorezca la clasificación en las clases con un número mayoritario de instancias. 

Una mejor métrica para nuestro dataset podría ser la *balanced accuracy* que se obtiene como la media aritmética entre *sensibilidad* y la *especificidad*. Recordemos que la *sensibilidad* (o *true positive rate*) se define como el porcentaje de patrones positivos predichos como positivos, mientras que la *especificidad* se define como el porcentaje de patrones negativos predichos como negativos. Su definición (en base a la matriz de confusión) es la que sigue:

$Sensiblidiad = \frac{TP}{TP + FN}$

$Especificidad = \frac{TN}{TN + FP}$

Si identificamos "1" como clase positiva y "0" como clase negativa tendríamos lo siguiente:

- Por una parte, nuestro modelo tendría una **sensibilidad alta** puesto que consigue muchos *true positives* y pocos *false negatives*. En el caso del ejemplo anterior, al tener 0 *false negatives*, tendríamos una sensibiliad de 1.
- Por otra parte, nuestro modelo tendría una **especificidad baja** puesto que el número de *true negatives* es bajo. En el caso del ejemplo anterior, al tener 0 true negatives, tendríamos una especificidad de 0.

Si calculamos *balanced accuracy* como:

$Balanced\_Acc = \frac{Sensiblidad + Especificidad}{2}$

Podemos ver como esta magnitud "penaliza" situaciones como la anterior, consiguiendo una *balanced accuracy* más próxima a 0.5 que a 1. En el caso propuesto más arriba, obtendríamos una *balanced accuracy* de 0.5 mostrando así la tendencia a clasificar todas las muestras como "1". Si bien esta métrica resulta adecuada para el problema que tratamos, se plantea una problemática relacionada con el uso de la función ``cross_val_score()``. Como se ha estudiado, esta función calcula la métrica especificada con la *keyword* ``scoring`` para cada una de las particiones del dataset. En caso de la validación *Leave-One-Out* únicamente se calcula una predicción, por lo que nuestra matriz de confusión únicamente tendrá un valor no nulo y eso podría generar operaciones no deseades al calcular métricas como la *sensibilidad* o la *especificidad*. Es más, la propia función ``cross_val_score()`` arroja un *warning* al tratar de calcularlas con *Leave-One-Out*.  A continuación, propongo un cálculo alternativo de la *matriz de confusión* consistente en almacenar las predicciones sobre todas las instancias.

In [3]:
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut, ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import random
import numpy as np

# Separamos variables de entrada X y variable objetivo Y
X = df[df.columns[:-1]]
Y = df['outcome']

# Incializamos una semilla para poder replicar el reparto
seed = random.randint(0, 10)

# Obtenemos los objetos de validación cruzada
kfold = KFold(n_splits = 10, random_state = seed, shuffle = True)
loocv = LeaveOneOut()
monte_carlo = ShuffleSplit(n_splits = 100, test_size = 0.3, random_state = seed)

# Generamos el modelo de KNeighbors
model_knn = KNeighborsClassifier()

# Entrenamos el modelo con los diferentes métodos de validación cruzada
res_kfold = cross_val_score(model_knn, X, Y, cv = kfold, scoring = 'balanced_accuracy')
# res_loocv = cross_val_score(model_knn, X, Y, cv = loocv, scoring = 'balanced_accuracy')
res_monte_carlo = cross_val_score(model_knn, X, Y, cv = monte_carlo, scoring = 'balanced_accuracy')

# Pasamos a obtener las métricas de LeaveOneOut()
y_true, y_pred = list(), list()
# Almacenamos el conjunto de entrada y sus etiquetas como arrays numpy
X_aux, y_aux = X.values, Y.values
# Obtenemos los diferentes splits que genera Leave-One-Out
for train, test in loocv.split(X_aux):
    # Calculamos el split (n-1 datos de entrenamiento, 1 de test)
    X_tr, X_te = X_aux[train, :], X_aux[test, :]
    y_tr, y_te = y_aux[train], y_aux[test]
    # Ajustamos el modelo
    model_knn.fit(X_tr, y_tr)
    # Calculamos la predicción con el elemento de test
    y_new = model_knn.predict(X_te)
    # Nos quedamos con el valor verdadero y con el valor predicho
    y_true.append(y_te[0])
    y_pred.append(y_new[0])
    
# Calculamos la matriz de confusión para un problema de clas. binaria
conf_mat = np.zeros(shape = (2, 2))
conf_mat[0, 0] = len(list(filter(lambda x: x[0] == x[1] == 1, zip(y_true, y_pred)))) # TP
conf_mat[0, 1] = len(list(filter(lambda x: x[0] != x[1] and x[1] == 1, zip(y_true, y_pred)))) # FP
conf_mat[1, 0] = len(list(filter(lambda x: x[0] != x[1] and x[1] == 0, zip(y_true, y_pred)))) # FN 
conf_mat[1, 1] = len(list(filter(lambda x: x[0] == x[1] == 0, zip(y_true, y_pred)))) # TN
# Mostramos la matriz obtenida
conf_mat

array([[492.,  46.],
       [  2.,   0.]])

In [4]:
# Calculamos sensibilidad y especificidad
tp, fp, fn, tn = conf_mat[0, 0], conf_mat[0, 1], conf_mat[1, 0], conf_mat[1, 1]
sens = tp / (tp + fn)
spec = tn / (tn + fp)
res_loocv = (sens + spec) / 2

In [5]:
print(f"Métricas KFOLD -> Balanced Accuracy Promedio: {res_kfold.mean()} \t Desv. estándar: {res_kfold.std()}")
print(f"Métricas LOOCV -> Balanced Accuracy: {res_loocv}") #\t Desv. estándar: {res_loocv.std()}")
print(f"Métricas MonteCarlo -> Balanced Accuracy Promedio: {res_monte_carlo.mean()} \t Desv. estándar: {res_monte_carlo.std()}")

Métricas KFOLD -> Balanced Accuracy Promedio: 0.49803846153846154 	 Desv. estándar: 0.003924019494610455
Métricas LOOCV -> Balanced Accuracy: 0.4979757085020243
Métricas MonteCarlo -> Balanced Accuracy Promedio: 0.49906148804991785 	 Desv. estándar: 0.002063295596979471


Como puede observarse de las métricas anteriores, nuestro clasificador parece seguir la tendencia del ejemplo de más arriba independientemente del método de validación cruzada utilizado. Vemos que la *balanced accuracy* toma un valor cercano a 0.5 en todos los casos, lo que indicaría una *sensibilidad* alta y una *especificidad* baja. Consideramos que esta métrica es más informativa que otras en este caso particular pues, si el modelo se utilizase para hacer predicciones sobre un conjunto de datos con un número elevado de instancias de clase "0", se obtendrían unos resultados poco satisfactorios.  

In [6]:
import sklearn
sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_